# Mitiq Tutorial

## Executors

Executors are python functions that consume a quantum circuit, and output an expectation value.
A type signature might look something like `Circuit -> float`.
That said, executors can have additional arguments used to control other parts of the execution process.
The circuit must be the first argument, however.

In [21]:
import cirq


def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

## ZNE

In [48]:
circuit = cirq.Circuit()
a, b, c = cirq.LineQubit.range(3)

circuit.append(cirq.H(a))
circuit.append(cirq.CNOT(a, b))
circuit.append(cirq.CNOT(a, c))

print(circuit)

0: ───H───@───@───
          │   │
1: ───────X───┼───
              │
2: ───────────X───


In [50]:
from qiskit import QuantumCircuit

qiskit_circuit = QuantumCircuit(3)
qiskit_circuit.h(0)
qiskit_circuit.cnot(0, 1)
qiskit_circuit.cnot(1, 2)
qiskit_circuit.s(0)

print(qiskit_circuit)

     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ S ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├──■──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘


In [51]:
from mitiq.zne.scaling import fold_global

print(fold_global(qiskit_circuit, 3))

     ┌───┐     ┌───┐┌─────┐     ┌───┐┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ S ├┤ Sdg ├──■──┤ H ├┤ H ├──■──┤ S ├
     └───┘┌─┴─┐└───┘└─────┘┌─┴─┐└───┘└───┘┌─┴─┐└───┘
q_1: ─────┤ X ├──■─────■───┤ X ├──────────┤ X ├──■──
          └───┘┌─┴─┐ ┌─┴─┐ └───┘          └───┘┌─┴─┐
q_2: ──────────┤ X ├─┤ X ├─────────────────────┤ X ├
               └───┘ └───┘                     └───┘


In [34]:
from mitiq import zne
from mitiq.benchmarks import generate_rb_circuits

circuit = generate_rb_circuits(n_qubits=2, num_cliffords=20)[0]

true_value = execute(circuit, noise_level=0.0)
noisy_value = execute(circuit)
zne_value = zne.execute_with_zne(circuit, execute)

print(f"Error w/o  Mitiq: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w Mitiq:    {abs((true_value - zne_value) / true_value):.3f}")

Error w/o  Mitiq: 0.582
Error w Mitiq:    0.325


## Calibration

In [26]:
import numpy as np
from mitiq import MeasurementResult


def execute_with_depolarizing(circuit, noise_level=0.001):
    circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    result = cirq.DensityMatrixSimulator().run(circuit, repetitions=100)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

In [46]:
# circuit.append(cirq.measure(circuit.all_qubits()))
# print(circuit)
# print(execute_with_depolarizing(circuit))
print(MeasurementResult(np.column_stack(list(cirq.DensityMatrixSimulator().run(circuit, repetitions=100).measurements.values()))))

MeasurementResult: {'nqubits': 100, 'qubit_indices': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), 'shots': 2, 'counts': {'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000': 2}}
MeasurementResult: {'nqubits': 2, 'qubit_indices': (0, 1), 'shots': 100, 'counts': {'00': 100}}


In [27]:
from mitiq import Calibrator

cal = Calibrator(execute_with_depolarizing, frontend="cirq")

cal.run(log=True)

| performance | circuit | method | extrapolation | scale factors | scale_method         |
| ----------- | ------- | ------ | ------------- | ------------- | -------------------- |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 2.0, 3.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 3.0, 5.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 2.0, 3.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 3.0, 5.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 2.0, 3.0 | fold_gates_at_random |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 3.0, 5.0 | fold_gates_at_random |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 2.0, 3.0 | fold_gates_at_random |
|     ✅      |   ghz   |  ZNE   | Linear        | 1.0, 3.0, 5.0 | fold_gates_at_random |
|     ❌      |    w    |  ZNE   | Richardson    | 1.0, 2.0, 3.0 | fold_global          |
|     ❌      |    w

In [28]:
cal.best_strategy()

{'technique': 'ZNE', 'factory': 'LinearFactory', 'scale_factors': [1.0, 2.0, 3.0], 'scale_method': 'fold_global'}